In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

# Algorithms used for modeling
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, RANSACRegressor, HuberRegressor, PassiveAggressiveRegressor
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
import xgboost as xgb
print('Algorithm packages imported!')

# Model selection packages used for sampling dataset and optimising parameters
from sklearn.model_selection import GridSearchCV
print('Model selection packages imported!')

import warnings
warnings.filterwarnings('ignore')

Algorithm packages imported!
Model selection packages imported!


In [2]:
def slicing(predata, target, slicing_index):
    data = predata.copy()
    y_ = data.pop(target).iloc[slicing_index:]
    new_ = data.iloc[:-slicing_index]
    new_['total_cases'] = y_.values
    return new_

def splitting(data, target, train_ratio):
    x_ = data.copy()
    y_ = x_.pop(target)
    if x_.shape[0] != y_.shape[0]:
        raise Exception('Dimension not match!')
        return
    elif train_ratio >= 1:
        raise Exception('Train Ratio > 1')
        return
    else:
        num_train_ = int(train_ratio * x_.shape[0])
        return x_.iloc[:num_train_], x_.iloc[num_train_:], y_.iloc[:num_train_], y_.iloc[num_train_:]

In [3]:
pre_sj = pd.read_excel('Data/SJ_transformed2.xlsx', index_col = 'Unnamed: 0')
pre_iq = pd.read_excel('Data/IQ_transformed2.xlsx', index_col = 'Unnamed: 0')
iq_pop = pd.read_csv('Data/iq_pop.csv')

In [4]:
sj = pre_sj[~pre_sj.total_cases.isnull()]
sj_valid = pre_sj[pre_sj.total_cases.isnull()]
iq = pre_iq[~pre_iq.total_cases.isnull()]
iq_valid = pre_iq[pre_iq.total_cases.isnull()]

# For-Loop Average Correlation

In [6]:
# San Juan
sj_corr_dict = {}
iq_corr_dict = {}
for i in range(1, 50):
    sj_corr_dict[i] = np.mean(slicing(sj, 'total_cases', i).corr()['total_cases'])
    iq_corr_dict[i] = np.mean(slicing(iq, 'total_cases', i).corr()['total_cases'])

## Feature Selection, select only features having correlation more than the average

In [7]:
def select_var(city, lag, corr_dict):
    avg_corr_ = corr_dict[lag]
    sliced_data_ = slicing(city, 'total_cases', lag)
    sliced_corr_ = sliced_data_.corr()['total_cases']
    selected_corr_ = [col for col in sliced_corr_.index if sliced_corr_.loc[col] > avg_corr_]
    return selected_corr_

# GridSearch

In [8]:
sj_models = [SVR(), KernelRidge(), ElasticNet(), Lasso(), GradientBoostingRegressor(), BayesianRidge(), 
          LassoLarsIC(), RandomForestRegressor(), xgb.XGBRegressor(), RANSACRegressor(), HuberRegressor()]


models = sj_models

SVR_param_grid = {'kernel':['linear', 'rbf']}
KR_param_grid = {'alpha': [0.1, 0.2, 0.3], 'coef0': [100], 'degree': [1, 2], 'gamma': [None], 
                 'kernel': ['linear']}
EN_param_grid = {'alpha': [0.001, 0.005, 0.1], 'copy_X': [True], 'l1_ratio': [0.3, 0.6, 0.8], 
                 'fit_intercept': [True], 'normalize': [False], 
                 'precompute': [False], 'max_iter': [300, 900], 
                 'tol': [0.0005, 0.001, 0.002], 'selection': ['random'], 
                 'random_state': [None]}
LASS_param_grid = {'alpha': [0.0005, 0.001, 0.005], 'copy_X': [True], 
                   'fit_intercept': [True], 'normalize': [False], 'precompute': [False], 
                   'max_iter': [300], 'tol': [0.01, 0.05, 0.1], 
                   'selection': ['random'], 'random_state': [None]}
GB_param_grid = {'loss': ['huber'], 'learning_rate': [0.01, 0.1, 0.3], 
                 'n_estimators': [300, 1000], 'max_depth': [3, 5], 
                 'min_samples_split': [0.0025, 0.005], 'min_samples_leaf': [3, 5, 7]}
BR_param_grid = {'n_iter': [200, 600], 'tol': [0.00001, 0.0001], 
                 'alpha_1': [0.00000001, 0.0000001, 0.000005], 
                 'alpha_2': [0.000005, 0.00001], 'lambda_1': [0.000005, 0.00001, 0.00005], 
                 'lambda_2': [0.00000001, 0.0000001], 'copy_X': [True]}
LL_param_grid = {'criterion': ['aic'], 'normalize': [True], 
                 'max_iter': [100, 500], 'copy_X': [True], 'precompute': ['auto'], 
                 'eps': [0.000001, 0.00001]}
RFR_param_grid = {'n_estimators': [50, 100, 200], 'max_features': ['auto'], 
                  'max_depth': [None, 2], 'min_samples_split': [5, 10], 
                  'min_samples_leaf': [2]}
XGB_param_grid = {'max_depth': [3], 'learning_rate': [0.1], 'n_estimators': [300], 
                  'booster': ['gbtree'], 'gamma': [0], 'reg_alpha': [0.1],
                  'reg_lambda': [0.7], 'max_delta_step': [0], 'min_child_weight': [1], 
                  'colsample_bytree': [0.5], 'colsample_bylevel': [0.2],
                  'scale_pos_weight': [1]}
RANSAC_param_grid = {'min_samples': [0.2, 0.5, 0.8], 'stop_probability': [0.1, 0.4, 0.8], 'max_skips': [10,30,50]}
HB_param_grid = {'epsilon':[1.1, 1.35, 1.7, 2], 'max_iter':[100,300,500], 'alpha':[0.00001, 0.0001, 0.005, 0.01],
                 'tol':[1e-5, 5e-5, 1e-4, 5e-4]}

sj_params_grid = [SVR_param_grid, KR_param_grid, EN_param_grid, LASS_param_grid, GB_param_grid, BR_param_grid, 
               LL_param_grid, RFR_param_grid, XGB_param_grid, RANSAC_param_grid, HB_param_grid]

params_grid = sj_params_grid

params_dict = {}
for i in range(len(models)):
    params_dict[models[i]] = params_grid[i]

In [9]:
def sj_error_cal(real, predict):
    real_real = [int(x) for x in np.expm1(real)]
    real_predict = [int(x) for x in np.expm1(predict)]
    return mean_absolute_error(real_predict, real_real)

def iq_error_cal(real, pre_predict, pop = iq_pop):
    if real.shape[0] != pre_predict.shape[0]:
        raise Exception("The shapes are not match with each other")
        return
    else:
        predict = pd.Series(pre_predict, index = real.index)
        for i in range(real.shape[0]):
            real.iloc[i] = real.iloc[i] * pop['Estimated_population'].iloc[np.argwhere(pop['Year'] == real.index.year[i])[0][0]] / 100000
            predict.iloc[i] = predict.iloc[i] * pop['Estimated_population'].iloc[np.argwhere(pop['Year'] == predict.index.year[i])[0][0]] / 100000
        return mean_absolute_error(real, predict)

In [10]:
def gridsearch_loop(city, models, params, x_train_, x_test_, y_train_, y_test_):
    results = list()
    if len(models) != len(params):
        raise Exception('No. of models is not equal to no. of params')
        return
    for algo in models:
        gridsearch = GridSearchCV(algo, param_grid = params[algo], scoring = 'neg_mean_absolute_error')
        gridsearch.fit(x_train_, y_train_)
        grid_best_ = gridsearch.best_estimator_
        algo_score = gridsearch.best_score_
        prediction_ = gridsearch.predict(x_test_)
        if city == 'SJ':
            test_score_ = sj_error_cal(y_test_, prediction_)
        elif city == 'IQ':
            test_score_ = iq_error_cal(y_test_, prediction_)
        else:
            raise Exception("Something is wrong!")
        results.append([grid_best_, -algo_score, test_score_])
    return results

STEP:
1. For each lag, find the useful features using select_var.
2. Split data to train and test using splitting(slicing(args.)).
3. For each algorithm, use gridsearch to perform GridSearch algorithm, input city, models, params, and result table.
4. Continuously iterate steps 1 and 2 with different lags

# Result Table

In [11]:
#all_result = pd.DataFrame(columns=['City', 'Lag', 'Training Size', 'No. of Features', 'Best Algorithm',
#                                   'Training Score', 'Testing Score'])
filename = 'Results/all_result_20200226_morning.csv'
all_result = pd.read_csv(filename)
all_result.shape

(352, 7)

In [15]:
all_result.tail()

,City,Lag,Training Size,No. of Features,Best Algorithm,Training Score,Testing Score
347,IQ,16,0.7,20,"LassoLarsIC(copy_X=True, criterion='aic', eps=...",1.643824,6.862141e+04
348,IQ,16,0.7,20,"RandomForestRegressor(bootstrap=True, ccp_alph...",1.598123,2.990783e+05
349,IQ,16,0.7,20,"XGBRegressor(base_score=0.5, booster='gbtree',...",1.801157,1.303585e+06
350,IQ,16,0.7,20,"RANSACRegressor(base_estimator=None, is_data_v...",1.499845,5.682507e+06
351,IQ,16,0.7,20,"HuberRegressor(alpha=0.005, epsilon=1.1, fit_i...",1.479605,2.477367e+07


## San Juan

In [16]:
import time
import warnings
warnings.filterwarnings('ignore')

In [17]:
city = 'SJ'
laglist = range(13,17)
cv_list = [0.7] #, 0.8]
for lag in laglist:
    print('Start step 1 of lag',lag)
    # Step 1
    #useful_features = select_var(sj, lag, sj_corr_dict)
    #n_features = len(useful_features)
    
    # Step 2
    for cv in cv_list:
        print('Start step 2 of cv',cv)
        x_train, x_test, y_train, y_test = splitting(slicing(sj, 'total_cases', lag), 'total_cases', cv)
        
        # Step 3
        print('Start step 3')
        #city, best_param, train_score, test_score = gridsearch_loop(city, models, params_dict, x_train, x_test, y_train, y_test)
        #all_result = all_result.append({'City': city, 'Lag': lag, 'Training Size': cv,
        #                                'No. of Features': n_features, 'Best Algorithm': best_param,
        #                                'Training Score': train_score, 'Testing Score': test_score}, ignore_index=True)
        results = gridsearch_loop(city, models, params_dict, x_train, x_test, y_train, y_test)
        for result in results:
            all_result = all_result.append({'City':city, 'Lag': lag, 'Training Size':cv, 'No. of Features': 20,
                                            'Best Algorithm': result[0], 'Training Score': result[1],
                                            'Testing Score': result[2]}, ignore_index=True)
        if cv == cv_list[-1] and lag == laglist[-1]:
            print('All iterations completed')
        else:
            print('Start cooling down')
            time.sleep(180)
            print('Cooling down complete') 
all_result.to_csv(filename, index = False)
print('The latest file has been already saved!')

Start step 1 of lag 13
Start step 2 of cv 0.7
Start step 3
[08:18:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:18:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:18:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:18:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:18:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:18:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Start cooling down
Cooling down complete
Start step 1 of lag 14
Start step 2 of cv 0.7
Start step 3
[08:35:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:35:38] WARNING: src/objec

In [105]:
#all_result.to_csv('Results/all_result_20200225_morning.csv', index = False)
all_result[all_result.City == 'SJ'].sort_values('Testing Score', ascending = True).head()

,City,Lag,Training Size,No. of Features,Best Algorithm,Training Score,Testing Score
46,SJ,3,0.7,20,"ElasticNet(alpha=0.1, copy_X=True, fit_interce...",0.816024,15.417857
68,SJ,4,0.7,20,"ElasticNet(alpha=0.1, copy_X=True, fit_interce...",0.804328,15.450000
128,SJ,7,0.7,20,"RandomForestRegressor(bootstrap=True, ccp_alph...",0.803831,15.612903
73,SJ,4,0.7,20,"RandomForestRegressor(bootstrap=True, ccp_alph...",0.841850,15.664286
117,SJ,6,0.7,20,"RandomForestRegressor(bootstrap=True, ccp_alph...",0.826544,15.673835


In [95]:
all_result.iloc[207]['Best Algorithm']

"RANSACRegressor(base_estimator=None, is_data_valid=None, is_model_valid=None,\n                loss='absolute_loss', max_skips=30, max_trials=100,\n                min_samples=0.2, random_state=None, residual_threshold=None,\n                stop_n_inliers=inf, stop_probability=0.8, stop_score=inf)"

In [ ]:
sj_algo = all_result[all_result.City == 'SJ'].sort_values('Testing Score', ascending = True)

NOTE (2020-02-23): Remove XGBoost, RFR, and SVR from the model list, since they gave relatively higher criteria (error).

# Test...

In [50]:
sub = pd.read_csv('Data/submission_format.csv')

In [58]:
# Iquitos
#iq_algo = all_result[all_result.City == 'IQ'].sort_values('Testing Score', ascending = True).iloc[0]['Best Algorithm']

In [96]:
iq_algo = RANSACRegressor(base_estimator=None, is_data_valid=None, is_model_valid=None,loss='absolute_loss', max_skips=30, max_trials=100,min_samples=0.2, random_state=None, residual_threshold=None, stop_probability=0.8)

In [51]:
iq_selected_vars = select_var(iq, 2, iq_corr_dict)
#iq_selected_vars

In [52]:
iq_lag = slicing(pre_iq[iq_selected_vars], 'total_cases', 2)

In [53]:
iq_lag_train, iq_lag_test = iq_lag[~iq_lag.total_cases.isnull()], iq_lag[iq_lag.total_cases.isnull()]

In [100]:
iq_algo.fit(iq_lag_train.drop('total_cases', axis = 1), iq_lag_train.total_cases)

RANSACRegressor(base_estimator=None, is_data_valid=None, is_model_valid=None,
                loss='absolute_loss', max_skips=30, max_trials=100,
                min_samples=0.2, random_state=None, residual_threshold=None,
                stop_n_inliers=inf, stop_probability=0.8, stop_score=inf)

In [101]:
iq_svr_result = iq_algo.predict(iq_lag_test.drop('total_cases', axis = 1))

In [102]:
iq_result = pd.Series(iq_svr_result, index = iq_lag_test.index)

In [103]:
for i in range(iq_result.shape[0]):
    #iq_result = real.iloc[i] * pop['Estimated_population'].iloc[np.argwhere(pop['Year'] == real.index.year[i])[0][0]] / 10e5
    iq_result.iloc[i] = iq_result.iloc[i] * iq_pop['Estimated_population'].iloc[np.argwhere(iq_pop['Year'] == iq_result.index.year[i])[0][0]] / 100000

In [104]:
iq_result = [int(max(result, 0)) for result in iq_result]

In [73]:
# San Juan
all_result[all_result.City == 'SJ'].sort_values('Testing Score', ascending = True).iloc[0]['Best Algorithm']

"ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.8,\n           max_iter=300, normalize=False, positive=False, precompute=False,\n           random_state=None, selection='random', tol=0.002, warm_start=False)"

In [111]:
sj_selected_vars = select_var(sj, 7, sj_corr_dict)

In [112]:
sj_algo = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',max_depth=2, max_features='auto', max_leaf_nodes=None,max_samples=None, min_impurity_decrease=0.0,min_impurity_split=None, min_samples_leaf=2,min_samples_split=5, min_weight_fraction_leaf=0.0,n_estimators=50, n_jobs=None, oob_score=False,random_state=None, verbose=0, warm_start=False)
sj_lag = slicing(pre_sj[sj_selected_vars], 'total_cases', 7)
sj_lag_train, sj_lag_test = sj_lag[~sj_lag.total_cases.isnull()], sj_lag[sj_lag.total_cases.isnull()]

In [113]:
sj_algo.fit(sj_lag_train.drop('total_cases', axis = 1), sj_lag_train.total_cases)
sj_preresult = sj_algo.predict(sj_lag_test.drop('total_cases', axis = 1))
sj_result = pd.Series(sj_preresult, index = sj_lag_test.index)

sj_result = [int(max(np.expm1(result), 0)) for result in sj_result]

In [114]:
sub.total_cases = np.hstack([sj_result, iq_result])

In [115]:
sub.to_csv('Results/SJlag7_RF_IQlag2_HB.csv', index = False)

# Debug Algorithm Name String Problem 

In [85]:
def name_debug(data, col = 'Best Algorithm', split_str = '\n    '):
    for i in range(data.shape[0]):
        data[col][i] = ''.join(data[col][i].split(split_str))

In [86]:
xx.head()

,City,Lag,Training Size,No. of Features,Best Algorithm,Training Score,Testing Score
0,SJ,1,0.7,20,"SVR(C=1.0, cache_size=200, coef0=0.0, degree=3...",0.840799,17.786477
1,SJ,1,0.7,20,"KernelRidge(alpha=0.3, coef0=100, degree=1, ga...",0.861460,18.697509
2,SJ,1,0.7,20,"ElasticNet(alpha=0.1, copy_X=True, fit_interce...",0.847556,15.989324
3,SJ,1,0.7,20,"Lasso(alpha=0.005, copy_X=True, fit_intercept=...",0.852662,17.120996
4,SJ,1,0.7,20,"GradientBoostingRegressor(alpha=0.9, ccp_alpha...",0.904697,18.818505


In [87]:
name_debug(xx)

AttributeError: 'SVR' object has no attribute 'split'

In [96]:
xx['Best Algorithm'][1] = ''.join(''.join(xx['Best Algorithm'][1].split('\n    ')).split('        '))

In [99]:
xx['Best Algorithm'][1]

"KernelRidge(alpha=0.3, coef0=100, degree=1, gamma=None, kernel='linear',kernel_params=None)"

In [ ]:
xx['Best Algorithm'][2] = ''.join(xx['Best Algorithm'])